In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_dir = "../../../DATA_MAY18/spec1d/"
fname = '2-8h30m-270-spec1d.npz'

In [ ]:
data = np.load(data_dir + fname) #object, flag (0, 1), wavelength. flags: 0 -> iamge, 1 -> ivar
type(data)

In [ ]:
data['data_ivar'].shape

In [ ]:
print(data['data_ivar'][0].shape)
print(data['data_ivar'][0][0].shape)
print(len(data['data_ivar']))

In [ ]:
collapsedSpectrum = data['data_ivar'][:, 0, :]
ivar = data['data_ivar'][:, 1, :]

In [ ]:
ivar.shape

In [ ]:
def ModelO2(wavelength_array, width, Amp = 1):
    """Returns the [OII] doublet model
    Parameters: wavelength_array: Full wavelength array of MMT BinoSpec. This is constant.
                width: width of the Gaussian doublets. This varies.
                Amp: Amplitude of the Gaussian doublets
    """
    
    Gaussian = lambda x, mean, std: np.exp(-((x - mean)/std)**2)
    
    #Values from http://classic.sdss.org/dr6/algorithms/linestable.html
    separation = (3729.875-3727.092)/2 #separation between lambda0 and the emission lines
    
    return Amp*(Gaussian(wavelength_array, lambda0-separation, std) + Gaussian(spectra, lambda0+separation, std))

In [9]:
def Window(wavelength_array, ngal, pixel_size, window_size_multiplier):
    """Returns windows to run the Model function over to speed up calculation
    Parameters: wavelength_array: Full wavelength array of MMT BinoSpec. This is constant
                ngal: number of galaxies in a given data file
                pixel_size: width of pixels in wavelength_array
                window_size_multiplier: Multiple this with pixel size to get width of window in wavelength_array space
    Returns: nwindow_ndarray: l x m x n ndarray where l = ngal, m = number of windows and 
                              n = (pixel_size*window_size_multiplier)//2 + 1
    """
    
    nwindow = len(wavelength_array) - window_size_multiplier//2 #number of windows per galaxy. 
                                                    #It is of this form b/c beyond this window exceeds the wavelength_array
    nwindow_array = np.zeros((nwindow, (pixel_size*window_size_multiplier)//2 + 1))
    
    #Generate nwindow windows of size ((pixel_size*window_size_multiplier)//2 + 1)
    for i in range(nwindow):
        nwindow_array[i] = np.arange(wavelength_array[i], wavelength_array[i] + pixel_size*(1 + window_size_multiplier), \
                                     pixel_size)
    
    print(nwindow_array[0])
    #Repeat nwindow_arary ngal times 
    #https://stackoverflow.com/questions/32171917/copy-2d-array-into-3rd-dimension-n-times-python
    nwindow_ndarray = np.repeat(nwindow_array[:, :, np.newaxis], ngal, axis=2)

    return nwindow_ndarray

## Synthetic data

In [ ]:
def Model(params, Amp = 1, std = 0.75): #Default, A = 1, std = 0.75
    """Returns the [OII] doublet model
    Parameters: Amp Amplitude
                params: tuple of spectra and lambda0 (centre point of the Model)
    """
    
    spectra, lambda0 = params
    Gaussian = lambda x, mean, std: np.exp(-((x - mean)/std)**2)
    
    #Values from http://classic.sdss.org/dr6/algorithms/linestable.html
    separation = (3729.875-3727.092)/2 #separation between lambda0 and the emission lines
    
    return Amp*(Gaussian(spectra, lambda0-separation, std) + Gaussian(spectra, lambda0+separation, std))

In [ ]:
wavelength_grid = np.arange(4500, 4550, .5) 
ngal = 64 #Number of galaxy in one data file

In [ ]:
synthSpectra = np.arange(4500, 5000, .5) #Note that spacing must be geq the minimum linewidth, i.e. 0.1
pureData = Model((synthSpectra, 4730), 1, .1)
errstd = 0.05
errData = np.random.normal(0, errstd, synthSpectra.shape)
synthData = pureData + errData

plt.subplot(2,1,1)
plt.plot(synthSpectra, synthData)
plt.plot(synthSpectra, pureData)
plt.xlabel("Wavelength", fontname = 'serif', fontsize = 15)
plt.ylabel("Flux", fontname = 'serif', fontsize = 15)

plt.subplot(2,1,2)
plt.plot(errData)
plt.xlabel("Wavelength", fontname = 'serif', fontsize = 15)
plt.ylabel("Flux Error", fontname = 'serif', fontsize = 15)

In [ ]:
def SNR_Calculatorv1(wavelength, flux, err):
    """Returns the SNR and lambda0 such that SNR is a function of lambda0
    Parameters: wavelength: array of wavelength range over which to test the filter
                flux: array of flux corresponding to the wavelength range
                err: array of flux error
                
    Returns: SNR: Signal-to-noise ratios of Amplitude
             lambda0: Different central values of the filter
    """
    
    lambda0_emitted = 3727.092 + (3729.875-3727.092)/2 #Midpoint of OII doublet
        
    #Initialise numpy arrays
    width = np.arange(0.1, 2.1, .1) #To calculate SNR at different linewidth
    z = np.zeros(len(wavelength))
    SNR = np.zeros((len(width), len(wavelength))) #width vs z grid
    dataPrime = flux/err #signal of data
    
    #Calculate SNR at different lambda0 and w
    for i in range(len(wavelength)):
        for j in range(len(width)):
            lambda0 = wavelength[i]
            modelFlux = Model((wavelength, lambda0), 1, width[j])
            modelPrime = modelFlux/err
            
            """A = (fluxPrime (dot) modelPrime)/(modelPrime (dot) modelPrime)
            sigmaA = 1/sqrt(modelPrime (dot) modelPrime)
            SNR = A/sigmaA"""
            sigmaA = 1./np.sqrt(np.dot(modelPrime, modelPrime))
            A = np.dot(dataPrime, modelPrime)/(sigmaA**(-2))
            SNR[j][i] = A/sigmaA
            
        #Convert lambda0 to z
        z[i] = lambda0/lambda0_emitted - 1
    
    return SNR, z, width

In [ ]:
import time

In [ ]:
t0 = time.time()
SNRv1, z, w = SNR_Calculatorv1(synthSpectra, synthData, errstd)
t1 = time.time()
totalv1 = t1-t0

In [ ]:
def SNR_Calculatorv2(wavelength, flux, err):
    """Returns the SNR and lambda0 such that SNR is a function of lambda0
    Parameters: wavelength: array of wavelength range over which to test the filter
                flux: array of flux corresponding to the wavelength range
                err: array of flux error
                
    Returns: SNR: Signal-to-noise ratios of Amplitude
             lambda0: Different central values of the filter
    """
    
    lambda0_emitted = 3727.092 + (3729.875-3727.092)/2 #Midpoint of OII doublet
        
    #Initialise numpy arrays
    width = np.arange(0.1, 2.1, .1) #To calculate SNR at different linewidth
    z = np.zeros(len(wavelength))
    SNR = np.zeros((len(width), len(wavelength))) #width vs z grid
    dataPrime = flux/err #signal of data
    #Calculate SNR at different lambda0 and w
    for i in range(len(wavelength)-50): 
        #The window is 100 px wide with centre at the 50th px. Hence we go upto nth-50 such that we do not have index error
        for j in range(len(width)):
            lambda0 = wavelength[i+50]
            modelFlux = Model((wavelength[i:i+100], lambda0), 1, width[j])
            modelPrime = modelFlux/err
            
            """A = (fluxPrime (dot) modelPrime)/(modelPrime (dot) modelPrime)
            sigmaA = 1/sqrt(modelPrime (dot) modelPrime)
            SNR = A/sigmaA"""
            sigmaA = 1./np.sqrt(np.dot(modelPrime, modelPrime))
            A = np.dot(dataPrime[i:i+100], modelPrime)/(sigmaA**(-2))
            SNR[j][i] = A/sigmaA
            
        #Convert lambda0 to z
        z[i] = lambda0/lambda0_emitted - 1
    
    return SNR, z, width

In [ ]:
t0 = time.time()
SNRv2, z, w = SNR_Calculatorv2(synthSpectra, synthData, errstd)
t1 = time.time()
totalv2 = t1-t0

In [ ]:
print(totalv1)
print(totalv2)

In [ ]:
import seaborn as sns

In [ ]:
plt.imshow(SNRv1, aspect = 'auto')

In [ ]:
np.where(SNRv1 == SNRv1.max())

In [ ]:
plt.imshow(SNRv2, aspect = 'auto')

In [ ]:
np.where(SNRv2 == SNRv2.max())

In [ ]:
SNRv2[0][410]

In [ ]:
SNRv1[0][460]